<a href="https://colab.research.google.com/github/Zia-Ul-Hasan/NLP_SEMACNTIC_PRODUCT_SEARCH/blob/main/EDA_semantic_product.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np


In [9]:
products_df = pd.read_csv('/content/product.csv', sep='\t')
queries_df = pd.read_csv('/content/query.csv',sep='\t')
labels_df = pd.read_csv('/content/label.csv',sep='\t')

In [11]:
print(products_df.info())
print(queries_df.info())
print(labels_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42994 entries, 0 to 42993
Data columns (total 9 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   product_id           42994 non-null  int64  
 1   product_name         42994 non-null  object 
 2   product_class        40142 non-null  object 
 3   category hierarchy   41438 non-null  object 
 4   product_description  36986 non-null  object 
 5   product_features     42994 non-null  object 
 6   rating_count         33542 non-null  float64
 7   average_rating       33542 non-null  float64
 8   review_count         33542 non-null  float64
dtypes: float64(3), int64(1), object(5)
memory usage: 3.0+ MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 480 entries, 0 to 479
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   query_id     480 non-null    int64 
 1   query        480 non-null    object
 2   qu

In [12]:
print(products_df.head())
print(queries_df.head())
print(labels_df.head())

   product_id                                       product_name  \
0           0                            solid wood platform bed   
1           1                        all-clad 7 qt . slow cooker   
2           2            all-clad electrics 6.5 qt . slow cooker   
3           3       all-clad all professional tools pizza cutter   
4           4  baldwin prestige alcott passage knob with roun...   

                  product_class  \
0                          Beds   
1                  Slow Cookers   
2                  Slow Cookers   
3  Slicers, Peelers And Graters   
4                    Door Knobs   

                                  category hierarchy  \
0  Furniture / Bedroom Furniture / Beds & Headboa...   
1  Kitchen & Tabletop / Small Kitchen Appliances ...   
2  Kitchen & Tabletop / Small Kitchen Appliances ...   
3                         Browse By Brand / All-Clad   
4  Home Improvement / Doors & Door Hardware / Doo...   

                                 product_de

In [19]:
merged_df = labels_df.merge(queries_df, on='query_id', how='left')

merged_df = merged_df.merge(products_df, on='product_id', how='left')

print(merged_df.head())

   id  query_id  product_id       label        query     query_class  \
0   0         0       25434       Exact  salon chair  Massage Chairs   
1   1         0       12088  Irrelevant  salon chair  Massage Chairs   
2   2         0       42931       Exact  salon chair  Massage Chairs   
3   3         0        2636       Exact  salon chair  Massage Chairs   
4   4         0       42923       Exact  salon chair  Massage Chairs   

                                        product_name  \
0       21.7 '' w waiting room chair with wood frame   
1                  22.5 '' wide polyester side chair   
2      24.4 '' w metal lounge chair with metal frame   
3  25 '' wide faux leather manual swivel standard...   
4  27.6 '' w antimicrobial leather seat waiting r...   

              product_class  \
0       Waiting Room Chairs   
1             Accent Chairs   
2  Reception Seating Chairs   
3                 Recliners   
4       Waiting Room Chairs   

                                  category 

In [26]:

merged_df['product_text'] = (
    merged_df['product_name'].fillna('') + ' ' +
    merged_df['product_description'].fillna('') + ' ' +
    merged_df['product_features'].fillna('')
)
print(merged_df.head())
print(merged_df.info())
print(merged_df.isnull().sum())
print(merged_df.isnull().sum().sum())

   id  query_id  product_id       label        query     query_class  \
0   0         0       25434       Exact  salon chair  Massage Chairs   
1   1         0       12088  Irrelevant  salon chair  Massage Chairs   
2   2         0       42931       Exact  salon chair  Massage Chairs   
3   3         0        2636       Exact  salon chair  Massage Chairs   
4   4         0       42923       Exact  salon chair  Massage Chairs   

                                        product_name  \
0       21.7 '' w waiting room chair with wood frame   
1                  22.5 '' wide polyester side chair   
2      24.4 '' w metal lounge chair with metal frame   
3  25 '' wide faux leather manual swivel standard...   
4  27.6 '' w antimicrobial leather seat waiting r...   

              product_class  \
0       Waiting Room Chairs   
1             Accent Chairs   
2  Reception Seating Chairs   
3                 Recliners   
4       Waiting Room Chairs   

                                  category 

In [30]:
cols_to_check = ['query_class', 'product_class', 'category hierarchy', 'product_description']

cleaned_df = merged_df.dropna(subset=cols_to_check).reset_index(drop=True)
print(cleaned_df.info())
print(cleaned_df.isnull().sum())
print(cleaned_df.isnull().sum().sum())
print(cleaned_df.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 181907 entries, 0 to 181906
Data columns (total 15 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   id                   181907 non-null  int64  
 1   query_id             181907 non-null  int64  
 2   product_id           181907 non-null  int64  
 3   label                181907 non-null  object 
 4   query                181907 non-null  object 
 5   query_class          181907 non-null  object 
 6   product_name         181907 non-null  object 
 7   product_class        181907 non-null  object 
 8   category hierarchy   181907 non-null  object 
 9   product_description  181907 non-null  object 
 10  product_features     181907 non-null  object 
 11  rating_count         145991 non-null  float64
 12  average_rating       145991 non-null  float64
 13  review_count         145991 non-null  float64
 14  product_text         181907 non-null  object 
dtypes: float64(3), in

PREPROCESSNG


In [31]:
merged_df['query'] = merged_df['query'].str.lower()
merged_df['product_text'] = merged_df['product_text'].str.lower()



import re

def clean_text(text):
    return re.sub(r'[^a-z0-9\s]', ' ', text)

merged_df['query'] = merged_df['query'].apply(clean_text)
merged_df['product_text'] = merged_df['product_text'].apply(clean_text)


merged_df['query'] = merged_df['query'].str.strip().str.replace(r'\s+', ' ', regex=True)
merged_df['product_text'] = merged_df['product_text'].str.strip().str.replace(r'\s+', ' ', regex=True)


In [32]:

!pip install -U sentence-transformers


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 36.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 38.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 19.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 58.3 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [33]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')


from tqdm import tqdm

# Convert to lists
queries = merged_df['query'].tolist()
products = merged_df['product_text'].tolist()

query_embeddings = model.encode(queries, convert_to_numpy=True, show_progress_bar=True, batch_size=64)
product_embeddings = model.encode(products, convert_to_numpy=True, show_progress_bar=True, batch_size=64)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.4k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/3648 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
merged_df['query_embedding'] = list(query_embeddings)
merged_df['product_embedding'] = list(product_embeddings)


In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

# Convert lists to NumPy arrays
query_vecs = np.vstack(merged_df['query_embedding'].values)
product_vecs = np.vstack(merged_df['product_embedding'].values)

# Compute cosine similarities
cos_similarities = np.array([
    cosine_similarity(query_vecs[i].reshape(1, -1), product_vecs[i].reshape(1, -1))[0][0]
    for i in range(len(query_vecs))
])

merged_df['cosine_similarity'] = cos_similarities


In [ ]:
# Map labels to binary for simplicity
label_map = {
    'Exact': 2,
    'Relevant': 1,
    'Irrelevant': 0
}
merged_df['label_numeric'] = merged_df['label'].map(label_map)
